# Tải các thư viện cần thiết

In [ ]:
import os
import pathlib
import glob2
import pandas as pd
import pickle
 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
 
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
import pickle
import time

# Biến toàn cục

In [ ]:
NUMBER_OF_DATA_POINT= 104000
NUMBER_OF_TEST=50000

# Mount drive và giải nén data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Tải dữ liệu

In [ ]:
!unzip /content/drive/MyDrive/zalo_ai_challange_data/Train-Test-Data_v2.zip
!unzip /content/public-test.zip
!unzip /content/dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/635-F-27/635-22.wav  
  inflating: dataset/635-F-27/635-17.wav  
  inflating: dataset/635-F-27/635-19.wav  
  inflating: dataset/635-F-27/635-6.wav  
  inflating: dataset/635-F-27/635-8.wav  
  inflating: dataset/635-F-27/635-5.wav  
  inflating: dataset/635-F-27/635-12.wav  
  inflating: dataset/635-F-27/635.wav  
  inflating: dataset/635-F-27/635-13.wav  
  inflating: dataset/635-F-27/635-15.wav  
  inflating: dataset/635-F-27/635-10.wav  
  inflating: dataset/635-F-27/635-7.wav  
  inflating: dataset/635-F-27/635-23.wav  
  inflating: dataset/635-F-27/635-24.wav  
  inflating: dataset/635-F-27/635-4.wav  
  inflating: dataset/635-F-27/635-20.wav  
  inflating: dataset/635-F-27/635-25.wav  
  inflating: dataset/635-F-27/635-18.wav  
  inflating: dataset/635-F-27/635-21.wav  
   creating: dataset/656-F-25/
  inflating: dataset/656-F-25/656-28.wav  
  inflating: dataset/656-F-25/656-5.wav  
  inflating: dataset/656

# Đọc thư mục dataset

In [ ]:
ls_dir = os.listdir("./dataset")

ls_all_audio_file = []
for name_dir in ls_dir:
    for path in glob2.glob('./dataset/{}/*wav'.format(name_dir)):
        ls_all_audio_file.append(path)
      
print("Số file âm thanh trong tập huấn luyện: ",len(ls_all_audio_file))
print("Số người trong tập huấn luyện: ", len(ls_dir))

Số file âm thanh trong tập huấn luyện:  10560
Số người trong tập huấn luyện:  400


# Tiền xử lí

## Các hàm cần thiết

In [ ]:
def decode_audio(audio_binary):
  return tf.audio.decode_wav(audio_binary)

def get_waveform(file_path):
  audio_binary = tf.io.read_file(file_path)
  return decode_audio(audio_binary)

def zeroPadding(data):
  zero_padding=tf.zeros([NUMBER_OF_DATA_POINT-data.shape[0],1],dtype=tf.float32)
  dt = tf.cast(data, tf.float32)  # Ép kiểu sang float32
  return tf.concat([dt, zero_padding], 0) # ghép 2 tensor lại với nhau

def rename(filePath):
  """
  từ dataset -> dataset2
  """
  return filePath[0:9]+'2'+filePath[9:]

def saveFileWav(data,sr,file_path):
  """
  """
  x=tf.reshape(data,[NUMBER_OF_DATA_POINT,1])       # chuyển từ đang (104000,) -> (104000,1)
  newFileWav=tf.audio.encode_wav(x,sr)              # mã hóa về file wav
  tf.io.write_file(file_path, newFileWav, name=None) # lưu file

def splitData(data,sr,file_path):
  num_iter=(int)((len(data)-1)/NUMBER_OF_DATA_POINT)+1 # Số file cần tách
  start=0
  end=NUMBER_OF_DATA_POINT
  for i in range(num_iter-1):
    # Tạo ra 1 file mới
    saveFileWav(data[start:end],sr,file_path[0:-4]+'_'+str(i)+".wav")
    # Cập nhật lại start và end
    start =end
    end = start+ NUMBER_OF_DATA_POINT
  # Trường hợp file còn dư thì thêm 0 vào
  saveFileWav(zeroPadding(data[start:]),sr,file_path[0:-4]+'_'+str(num_iter-1)+".wav")

def normalize(filePath):
  # Đọc dữ liệu từ file
  data, sr=get_waveform(filePath)
  # Tạo đường dẫn mới
  newFilePath=rename(filePath)
  # Nếu độ dài của data nhỏ hơn 104000
  n=len(data)
  if (n<=104000):
    saveFileWav(zeroPadding(data),sr,newFilePath) # Lưu vào thư mục mới
  # Nếu độ dài lớn hơn 104000
  else:
    splitData(data,sr,newFilePath)

## Tách các file âm thanh thành các file có cùng kích thước

In [ ]:
for i in range(len(ls_all_audio_file)):
  try: 
    normalize(ls_all_audio_file[i])
  except Exception as es:
    None

KeyboardInterrupt: ignored

In [ ]:
#Download thư mục dataset2 xuống
!zip -r /content/dataset2.zip /content/dataset2
from google.colab import files
files.download("/content/dataset2.zip")

In [ ]:
# Xóa thư mục dataset để tránh lãng phí tài nguyên
# !rm -rf /content/dataset

# Tạo cơ sở dữ liệu

## Các hàm cần thiết 

In [ ]:
def ls_file_in_dir(dir):
  """
  Lấy danh sách các file trong thư mục dir
  ----
  dir: string
       Địa chỉ thư mục cần lấy danh sách

  ----
  Output: list
          Địa chỉ các file trong thư mục
  """
  ls_file = []
  for path in glob2.glob(dir+'/*wav'):
    ls_file.append(path)
  return ls_file

def create_df_dataset(ls_all_audio_file,ls_dir,max_len = 10):
    '''
    Parameters:
        max_len: Số lượng phần tử trong tập dataset
                type: int
        ls_all_audio_file: danh sách tất cả các file audio
                type: list
        ls_dir: danh sách các thư mục
                type: list
    return:
        Dataframe
    '''
    i = 0
    df_train = pd.DataFrame(columns=['Dir_audio_1', 'Dir_audio_2', 'Label'])
    while True:
        if i == int(max_len/2):
            break
        i+=1
        pair_audio = np.random.choice(ls_all_audio_file, 2)

        audio_dir_1 =pair_audio[0].split('\\')[0]
        audio_dir_2 =pair_audio[1].split('\\')[0]

        if audio_dir_1.split('/')[2] == audio_dir_2.split('/')[2]:
            label = 1
        else:
            label = 0

        df_train = df_train.append({'Dir_audio_1': pair_audio[0],
                        'Dir_audio_2': pair_audio[1],
                        'Label': label},
                         ignore_index = True)
        
    for i in range(int(max_len/2)):
        rd_person = np.random.choice(ls_dir, 1)[0]
        dir_name = './dataset2/'+rd_person
        ls_file = ls_file_in_dir(dir_name)
        pair_audio = np.random.choice(ls_file, 2)
        au1 = pair_audio[0]
        au2 = pair_audio[1]

        df_train = df_train.append({'Dir_audio_1': au1,
                        'Dir_audio_2': au2,
                        'Label': 1},
                        ignore_index = True)
    return df_train.sample(frac=1)

## Tạo các đường dẫn cho tập huấn luyện mới

In [ ]:
ls_dir_2 = os.listdir("./dataset2")

ls_all_audio_file_2 = []
for name_dir in ls_dir_2:
    for path in glob2.glob('./dataset2/{}/*wav'.format(name_dir)):
        ls_all_audio_file_2.append(path)
      
print("Số file âm thanh trong tập huấn luyện mới: ",len(ls_all_audio_file_2))
print("Số người trong tập huấn luyện mới: ", len(ls_dir_2))

Số file âm thanh trong tập huấn luyện mới:  19240
Số người trong tập huấn luyện mới:  398


In [ ]:
# Tạo nhãn
NUMBER_OF_SAMPLE= 25000
df_train = create_df_dataset(ls_all_audio_file_2,ls_dir_2, NUMBER_OF_SAMPLE)
df_train

,Dir_audio_1,Dir_audio_2,Label
19184,./dataset2/495-M-35/495-37.wav,./dataset2/495-M-35/495-31_0.wav,1
10891,./dataset2/791-F-26/791-4_3.wav,./dataset2/624-M-35/624-18_0.wav,0
6960,./dataset2/585-F-28/585-26_2.wav,./dataset2/797-F-28/797-15_1.wav,0
12143,./dataset2/453-F-27/453-2_1.wav,./dataset2/532-M-27/532-3_1.wav,0
4446,./dataset2/566-F-32/566-20_1.wav,./dataset2/671-M-25/671-18_1.wav,0
...,...,...,...
20349,./dataset2/442-M-36/442-6_1.wav,./dataset2/442-M-36/442-34_3.wav,1
17415,./dataset2/51-F-30/51-9_0.wav,./dataset2/51-F-30/51-18_2.wav,1
23121,./dataset2/668-M-29/668-16_1.wav,./dataset2/668-M-29/668-14.wav,1
20608,./dataset2/734-M-41/734-3_2.wav,./dataset2/734-M-41/734-16_1.wav,1


# MFCC

In [ ]:
# Tách cột
ls_audio1=df_train["Dir_audio_1"].values
ls_audio2=df_train["Dir_audio_2"].values

In [ ]:
def get_mfccs(audio_file=None ,
              signals=None, 
              sample_rate = 44100,
              num_mfccs = 39,
              frame_length=1024,
              frame_step=512, 
              fft_length=1024):
    """Compute the MFCCs for audio file
    
    Keyword Arguments:
        audio_file {str} -- audio wav file path (default: {None})
        signals {tensor} -- input signals as tensor or np.array in float32 type (default: {None})
        sample_rate {int} -- sampling rate (default: {44100})
        num_mfccs {int} -- number of mfccs to keep (default: {39})
        frame_length {int} -- frame length to compute STFT (default: {1024})
        frame_step {int} -- frame step to compute STFT (default: {512})
        fft_length {int} -- FFT length to compute STFT (default: {1024})
    
    Returns:
        Tensor -- mfccs as tf.Tensor
    """

    
    if signals is None and audio_file is not None:
      audio_binary = tf.io.read_file(audio_file)
      # tf.contrib.ffmpeg not supported on Windows, refer to issue
      # https://github.com/tensorflow/tensorflow/issues/8271
      waveform, sample_rate = tf.audio.decode_wav(audio_binary)
      signals = tf.reshape(waveform, [1, -1])
    
    # Step 1 : signals->stfts
  
    stfts = tf.signal.stft(signals, frame_length=frame_length, frame_step=frame_step,
                                   fft_length=fft_length)
    
    spectrograms = tf.abs(stfts)

    num_spectrogram_bins = stfts.shape[-1]

    lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80

    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                                            num_mel_bins, 
                                            num_spectrogram_bins, 
                                            sample_rate,
                                            lower_edge_hertz,
                                            upper_edge_hertz)

    mel_spectrograms = tf.tensordot( spectrograms, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    # Compute MFCCs from log_mel_spectrograms and take the first 13.
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(
    log_mel_spectrograms)[..., :num_mfccs]
    
    return mfccs

## Chuyển tập huấn luyện thành dạng MFCC

In [ ]:
%%time
dataset=[]
for i in range(len(ls_audio1)):
  mfcc1= get_mfccs(ls_audio1[i])

  mfcc2= get_mfccs(ls_audio2[i])
  # Ghép 2 vector 
  mfcc = np.concatenate((mfcc1,mfcc2))
  # Thêm vào dataset
  dataset.append(mfcc)

CPU times: user 14min 37s, sys: 1min 36s, total: 16min 14s
Wall time: 14min 43s


In [ ]:
# # Save data
# file = open('/content/drive/MyDrive/zalo_ai_challange_data/train_data_2.pkl', 'wb')

# pickle.dump(dataset, file)
# file.close

# # Save y label
# y = df_train['Label'].values.astype('float32')
# file = open('/content/drive/MyDrive/zalo_ai_challange_data/label_2.pkl', 'wb')
# pickle.dump(y, file)
# file.close

<function BufferedWriter.close>

# Áp dụng một mô hình đơn giản để thực thi

## Xây dựng mô hình

In [ ]:
# file = open('/content/drive/MyDrive/zalo_ai_challange_data/train_data_2.pkl', 'rb')

# dataset = pickle.load(file)


In [ ]:
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from keras.optimizers import SGD

In [ ]:
input_layer = Input(shape=(202,78,1))

# Tầng 1
conv_layer1= Conv2D(64,(3,3),padding="same",activation="relu")(input_layer)

# Tầng 2
conv_layer2= Conv2D(64,(3,3),padding="same",activation="relu")(conv_layer1)

# pooling
pooling_layer1=MaxPooling2D()(conv_layer2)

#Tầng 3
conv_layer3= Conv2D(128,(3,3),padding="same",activation="relu")(pooling_layer1)

# Tầng 4
conv_layer4= Conv2D(128,(3,3),padding="same",activation="relu")(conv_layer3)

# Tầng pooling
pooling_layer2=MaxPooling2D()(conv_layer4)

# Tầng 5
conv_layer5=Conv2D(256,(3,3),padding="same",activation="relu")(pooling_layer2)

# Tầng 6
conv_layer6=Conv2D(256,(3,3),padding="same",activation="relu")(conv_layer5)

# Tầng 7
conv_layer7=Conv2D(256,(3,3),padding="same",activation="relu")(conv_layer6)
# Tầng 8
conv_layer8=Conv2D(256,(3,3),padding="same",activation="relu")(conv_layer7)

# pooling
pooling_layer3=MaxPooling2D()(conv_layer8)

# 3 tầng 512
conv_layer8=Conv2D(512,(3,3),padding="same",activation="relu")( pooling_layer3)

conv_layer9=Conv2D(512,(3,3),padding="same",activation="relu")( conv_layer8)

conv_layer10 =Conv2D(512,(3,3),padding="same",activation="relu")( conv_layer9)

conv_layer11 =Conv2D(512,(3,3),padding="same",activation="relu")( conv_layer10)

pooling_layer4 =MaxPooling2D()(conv_layer11)

# 3 tầng 512

conv_layer12=Conv2D(512,(3,3),padding="same",activation="relu")( pooling_layer4)

conv_layer13=Conv2D(512,(3,3),padding="same",activation="relu")( conv_layer12)

conv_layer14 =Conv2D(512,(3,3),padding="same",activation="relu")( conv_layer13)

conv_layer15 =Conv2D(512,(3,3),padding="same",activation="relu")( conv_layer14)

pooling_layer5 =MaxPooling2D()(conv_layer15)

# Flatten
flatten_layer=Flatten()(pooling_layer5)

# Dense layer
dense_layer1=Dense(1000,activation="relu")(flatten_layer)

dense_layer2=Dense(100,activation="relu")(dense_layer1)

# Output
out_put=Dense(1,activation="sigmoid")(dense_layer2)

# tiến hành huấn luyện
model=Model(inputs = input_layer, outputs=out_put)
#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

from keras.optimizers import SGD
opt = SGD(lr=0.005)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=["accuracy"])

model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 202, 78, 1)]      0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 202, 78, 64)       640       
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 202, 78, 64)       36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 101, 39, 64)       0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 101, 39, 128)      73856     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 101, 39, 128)      147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 50, 19, 128)      

## One - hot label

In [ ]:
# y = df_train['Label'].values.astype('float32')
dataset = np.array(dataset)

## Chia tập train - validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size = 0.2, random_state = 0, stratify = y)

X_train = X_train.reshape(X_train.shape[0], 202, 78,1)
X_test = X_test.reshape(X_test.shape[0], 202, 78, 1)


print(X_train.shape)
print(X_test.shape)
print(np.sum(y_train))

(20000, 202, 78, 1)
(5000, 202, 78, 1)
10030.0


## Huấn luyện và ghi trọng số lại

In [ ]:
filepath = "/content/save_weights/final_weights.h5"

call_back = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="max",
)

if os.path.exists(filepath):
    model.load_weights(filepath)


H=model.fit(X_train,
            y_train,
            validation_data=(X_test, y_test),
            epochs=20,
            batch_size=32,
            callbacks=[call_back]
            )

Epoch 1/20
625/625 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5057
Epoch 00001: val_accuracy improved from -inf to 0.50640, saving model to /content/save_weights/final_weights.h5
625/625 [==============================] - 132s 211ms/step - loss: 0.6931 - accuracy: 0.5057 - val_loss: 0.6931 - val_accuracy: 0.5064
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5081
Epoch 00002: val_accuracy did not improve from 0.50640
625/625 [==============================] - 131s 210ms/step - loss: 0.6930 - accuracy: 0.5081 - val_loss: 0.6931 - val_accuracy: 0.5056
Epoch 3/20
625/625 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5086
Epoch 00003: val_accuracy did not improve from 0.50640
625/625 [==============================] - 132s 210ms/step - loss: 0.6928 - accuracy: 0.5086 - val_loss: 0.6930 - val_accuracy: 0.5020
Epoch 4/20
625/625 [==============================] - ETA: 0s - loss: 0.6927 - accura

# Kiểm tra trên tập đánh giá

---



In [ ]:
# Đọc file csv
public_test=pd.read_csv("/content/public-test.csv")
# Truy cập vào file 
audio_1_path="/content/public-test/" + public_test["audio_1"].values 
audio_2_path="/content/public-test/" + public_test["audio_2"].values 

In [ ]:
# Viết lại hàm chuẩn hóa
def splitData(data,res):
  num_iter=(int)((len(data)-1)/NUMBER_OF_DATA_POINT)+1 # Số file cần tách
  start=0
  end=NUMBER_OF_DATA_POINT
  for i in range(num_iter-1):
    # Thêm vào biến kết quả
    res.append(data[start:end])
    # Cập nhật lại start và end
    start =end
    end = start+ NUMBER_OF_DATA_POINT
  # Trường hợp file còn dư thì thêm 0 vào
  res.append(zeroPadding(data[start:]))

def normalize(filePath):
  """
  
  ---- OUTPUT----
  + Các mảng 1 chiều đã được chuẩn hóa
  type(list)
  + sampleRate
  """
  res=[]
  # Đọc dữ liệu từ file
  data, sampleRate=get_waveform(filePath)
  # Nếu độ dài của data nhỏ hơn 104000
  if (len(data)<=104000):
    res.append(zeroPadding(data))
  # Nếu độ dài lớn hơn 104000
  else:
    splitData(data,res)
  return res,sampleRate

def get_test(path1, path2):
  x,sr1=normalize(path1)
  y,sr2=normalize(path2)

  x_mfcc=[]
  for i in range(len(x)):
    x_mfcc.append(get_mfccs(signals=tf.reshape(x[i],[1,-1]), sample_rate=sr1))

  y_mfcc=[]
  for i in range(len(y)):
    y_mfcc.append(get_mfccs(signals=tf.reshape(y[i],[1,-1]), sample_rate=sr1))
  new_shape=(x_mfcc[0].shape[1], x_mfcc[0].shape[2], -1)
  res=[]
  for i in range(len(x)):
    for j in range(len(y)):
      res.append(np.concatenate((x_mfcc[0], y_mfcc[0])).reshape(new_shape))

  return res

def getLabel(x):
  if x>0.5:
    return 1
  return 0

def output(test):
  # Tiến hành predict
  s=0.0
  N=len(test)
  for j in range(N):
    s+=model.predict(np.expand_dims(test[j],0))[0]
  return getLabel(s/N)
  

## Tạo cơ sở dữ liệu cho tập test

In [ ]:
from google.colab import files
# open a file, where you ant to store the data

file = open('testset1', 'wb')

testset=[]
for i in range(NUMBER_OF_TEST):
  testset.append(get_test1(ls_audio1[i], ls_audio2[i]))
# dump information to that file
pickle.dump(testset, file)

# close the file
file.close()
files.download("/content/testset1")


# Predict

## Predict khi nộp bài

In [ ]:
label=[]
# N=1000
# s=0
for i in range(NUMBER_OF_TEST):
  # start_time = time.time()
  test=get_test1(ls_audio1[i], ls_audio2[i])
  label.append(output(test))
  # end_time=time.time()
  # s+=(end_time-start_time)
# print(s/N)

## Predict khi test mô hình

In [ ]:
label=[]
for i in range(NUMBER_OF_TEST):
  label.append(output(test[i]))

# Ghi file csv